In [1]:
#Get the customer first name ,customer last name, order id , order status where it should meet the two conditions a) order id should be less than 1000 and order date should be less than 2013-08-25  b)order status should be pending status. sort the data order id descending .

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

In [3]:
import pyspark.sql.functions as f

In [4]:
customers_csv= spark.read.format('csv').options(header='True',inferSchema='true').load('/FileStore/tables/customers.csv')

In [5]:
cust=customers_csv.select(
f.split(customers_csv.column_name,',')[0].cast('int').alias('customer_id'),
f.split(customers_csv.column_name,',')[1].cast('string').alias('customer_fname'),
f.split(customers_csv.column_name,',')[2].cast('string').alias('customer_lname'),
f.split(customers_csv.column_name,',')[3].cast('string').alias('customer_email'),
f.split(customers_csv.column_name,',')[4].cast('string').alias('customer_password'),
f.split(customers_csv.column_name,',')[5].cast('string').alias('customer_street'),
f.split(customers_csv.column_name,',')[6].cast('string').alias('customer_city'),
f.split(customers_csv.column_name,',')[7].cast('string').alias('customer_state'),
f.split(customers_csv.column_name,',')[8].cast('string').alias('customer_zipcode')
)

In [6]:
cust.show()

In [7]:
orders_csv = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/orders.csv')
orders=orders_csv.select(
f.split(orders_csv.column_name,',')[0].cast('int').alias('order_id'),
f.split(orders_csv.column_name,',')[1].cast('date').alias('order_date'),
f.split(orders_csv.column_name,',')[2].cast('int').alias('order_customer_id'),
f.split(orders_csv.column_name,',')[3].cast('string').alias('order_status')
)

In [8]:
orders.show()

In [9]:
orders.join(cust, cust.customer_id==orders.order_customer_id,"inner").\
filter((orders.order_date<'2013-08-25') & (orders.order_id<1000) | (orders.order_status=='PENDING')).\
select(cust.customer_fname,cust.customer_lname,orders.order_id,orders.order_status).sort(f.desc("order_id")).show()